本章将介绍卷积神经网络，也叫  convnet，它是计算机视觉应用几乎都在使用的一种深度学习模型。你将学到将卷积神经网络应用于图像分类问题，特别是那些训练数据集较小的问题。如果你工作的地方并非大型科技公司，这也将是你最常见的使用场景。   

李宏毅2020 Convolutional Neural Network part1


### 5.1　卷积神经网络简介

我们将深入讲解卷积神经网络的原理，以及它在计算机视觉任务上为什么如此成功。但在此之前，我们先来看一个简单的卷积神经网络示例，即使用卷积神经网络对 MNIST数字进行分类，这个任务我们在第 2章用密集连接网络做过（当时的测试精度为   97.8%）。虽然本例中的卷积神经网络很简单，但其精度肯定会超过第 2章的密集连接网络。

下列代码将会展示一个简单的卷积神经网络。它是Conv2D层和MaxPooling2D层的堆叠。很快你就会知道这些层的作用。

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [26]:
model = models.Sequential()
#previous model
#model = keras.Sequential([
#    layers.Dense(512, activation="relu"),
#    layers.Dense(10, activation="softmax")
#])

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) #Conv2D层 28-3+1=26
model.add(layers.MaxPooling2D((2, 2))) #MaxPooling2D 26/2=13
model.add(layers.Conv2D(64, (3, 3), activation='relu')) #13-3+1=11
model.add(layers.MaxPooling2D((2, 2))) #11/2=5
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 5-3+1=3

重要的是，卷积神经网络接收形状为(image_height, image_width, image_channels)的输入张量（不包括批量维度）。本例中设置卷积神经网络处理大小为(28, 28, 1)的输入张量，这正是 MNIST图像的格式。我们向第一层传入参数input_shape=(28,  28, 1)来完成此设置。

In [27]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


可以看到，每个Conv2D层和MaxPooling2D层的输出都是一个形状为(height,   width,channels)的  3D张量。宽度和高度两个维度的尺寸通常会随着网络加深而变小。通道数量由传入Conv2D层的第一个参数所控制（32或   64）。

下一步是将最后的输出张量［大小为 (3, 3, 64)］输入到一个密集连接分类器网络中，即Dense层的堆叠，你已经很熟悉了。这些分类器可以处理  1D向量，而当前的输出是 3D张量。

首先，我们需要将 3D输出展平为 1D，然后在上面添加几个Dense层

In [29]:
#如你所见，在进入两个Dense层之前，形状(3,  3, 64)的输出被展平为形状(576,)的向量。
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu')) # 导入 (3，3，64)
model.add(layers.Dense(10, activation='softmax')) # 最终导出成10维

In [30]:
model.summary() #如你所见，在进入两个Dense层之前，形状(3,  3, 64)的输出被展平为形状(576,)的向量。

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)              

In [32]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 20s 21ms/step - loss: 1.2931 - accuracy: 0.6009
Epoch 2/5
938/938 [==============================] - 20s 21ms/step - loss: 0.0915 - accuracy: 0.9789
Epoch 3/5
938/938 [==============================] - 20s 21ms/step - loss: 0.0497 - accuracy: 0.9882
Epoch 4/5
938/938 [==============================] - 20s 21ms/step - loss: 0.0343 - accuracy: 0.9915
Epoch 5/5
938/938 [==============================] - 20s 21ms/step - loss: 0.0289 - accuracy: 0.9926


In [33]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 1s 2ms/step - loss: 0.0355 - accuracy: 0.9909


0.9908999800682068

第 2章密集连接网络的测试精度为    97.8%，但这个简单卷积神经网络的测试精度达到了99.3%，我们将错误率降低了 68%（相对比例）。相当不错！

与密集连接模型相比，为什么这个简单卷积神经网络的效果这么好？要回答这个问题，我们来深入了解Conv2D层和MaxPooling2D层的作用。

### 5.1.1　卷积运算
密集连接层和卷积层的根本区别在于，  Dense层从输入特征空间中学到的是全局模式（比如对于 MNIST数字，全局模式就是涉及所有像素的模式），而卷积层学到的是局部模式respective filed（见图 5-1），对于图像来说，学到的就是在输入图像的二维小窗口中发现的模式。在上面的例子中，这些窗口的大小都是 3×3。  

这个重要特性使卷积神经网络具有以下两个有趣的性质。

1. 卷积神经网络学到的模式具有平移不变性（translation invariant）。卷积神经网络在图像右下角学到某个模式之后，它可以在任何地方识别这个模式，比如左上角。对于密集连接网络来说，如果模式出现在新的位置，它只能重新学习这个模式。这使得卷积神经网络在处理图像时可以高效利用数据（因为视觉世界从根本上具有平移不变性），它只需要更少的训练样本就可以学到具有泛化能力的数据表示。   

每一组respective filed都有对应的一组neuron，那么对于不同位置的同一识别模块，能否在对应的respective filed中共享参数呢?  
可以，公用的一组参数就叫filter 1


2. 卷积神经网络可以学到模式的空间层次结构（spatial  hierarchies of patterns），见图 5-2。第一个卷积层将学习较小的局部模式（比如边缘），第二个卷积层将学习由第一层特征组成的更大的模式，以此类推。这使得卷积神经网络可以有效地学习越来越复杂、越来越抽象的视觉概念（因为视觉世界从根本上具有空间层次结构）。  

对于包含两个空间轴（高度和宽度）和一个深度轴（也叫通道轴）的    3D张量，其卷积也叫特征图（feature map）。对于 RGB图像，深度轴的维度大小等于  3，因为图像有 3个颜色通道：红色、绿色和蓝色。对于黑白图像（比如 MNIST数字图像），深度等于  1（表示灰度等级）。卷积运算从输入特征图中提取图块，并对所有这些图块应用相同的变换，生成输出特征图（output feature map）。该输出特征图仍是一个 3D张量，具有宽度和高度，其深度可以任意取值，因为输出深度是层的参数，深度轴的不同通道不再像 RGB输入那样代表特定颜色，而是代表过滤器（filter）。过滤器对输入数据的某一方面进行编码，比如，单个过滤器可以从更高层次编码这样一个概念：“输入中包含一张脸。”

MNIST示例中，第一个卷积层接收一个大小为(28,  28, 1)的特征图，并输出一个大小为(26,  26, 32)的特征图，即它在输入上计算  32个过滤器。对于这  32个输出通道，每个通道都包含一个 26×26的数值网格，它是过滤器对输入的响应图（response   map），表示这个过滤器模式在输入中不同位置的响应（见图  5-3）。这也是特征图这一术语的含义：深度轴的每个维度都是一个特征（或过滤器），而 2D张量output[:,  :, n]是这个过滤器在输入上的响应的二维空间图（map）。  

卷积由以下两个关键参数所定义。

从输入中提取的图块尺寸：这些图块的大小通常是 3×3或 5×5。本例中为 3×3，这是很常见的选择。  
输出特征图的深度：卷积所计算的过滤器的数量。本例第一层的深度为 32，最后一层的深度是 64。   

对于 Keras的Conv2D层，这些参数都是向层传入的前几个参数：Conv2D(output_depth,(window_height, window_width))。  

卷积convolution的工作原理：在 3D输入特征图上滑动（slide）这些  3×3或  5×5的窗口，在每个可能的位置停止并提取周围特征的 3D图块［形状为(window_height,  window_width, input_depth)］。然后每个 3D图块与学到的同一个权重矩阵［叫作卷积核（convolution     kernel）］做张量积，转换成形状为 (output_depth,)的   1D向量。然后对所有这些向量进行空间重组，使其转换为形状为 (height, width, output_depth)的   3D输出特征图。输出特征图中的每个空间位置都对应于输入特征图中的相同位置（比如输出的右下角包含了输入右下角的信息）。举个例子，利用 3×3的窗口，向量output[i,  j,  :]来自 3D图块input[i-1:i+1,j-1:j+1, :]。整个过程详见图 5-4。   

注意，输出的宽度和高度可能与输入的宽度和高度不同。不同的原因可能有两点。   

边界效应，可以通过对输入特征图进行填充来抵消。  #如果移动后超出了边界，可以通过对边界填充 padding
使用了步幅（stride），稍后会给出其定义。  #每次移动的距离  

为了对特征图进行下采样，我们不用步幅，而是通常使用最大池化（max-pooling）运算，你在第一个卷积神经网络示例中见过此运算。下面我们来深入研究这种运算。

### 5.1.2　最大池化运算
在卷积神经网络示例中，你可能注意到，在每个 MaxPooling2D层之后，特征图的尺寸都会减半。例如，在第一个MaxPooling2D层之前，特征图的尺寸是   26×26，但最大池化运算将其减半为 13×13。这就是最大池化的作用：对特征图进行下采样，与步进卷积类似。

最大池化是从输入特征图中提取窗口，并输出每个通道的最大值。   

它的概念与卷积类似，但是最大池化使用硬编码的max张量运算对局部图块进行变换，而不是使用学到的线性变换（卷积核）。最大池化与卷积的最大不同之处在于，最大池化通常使用  2×2的窗口和步幅  2，其目的是将特征图下采样 2倍。与此相对的是，卷积通常使用  3×3窗口和步幅  1。

为什么要用这种方式对特征图下采样？为什么不删除最大池化层，一直保留较大的特征图？我们来这么做试一下。这时模型的卷积基（convolutional base）如下所示。  

但pooling不可避免的会损失掉部分信息，随着运算能力的加强，pooling又被抛弃的趋势。


In [38]:
# 不运行 pooling
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [41]:
model_no_max_pool.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


这种架构有什么问题？有如下两点问题。

这种架构不利于学习特征的空间层级结构。第三层的       3×3窗口中只包含初始输入的7×7窗口中所包含的信息。卷积神经网络学到的高级模式相对于初始输入来说仍然很小，这可能不足以学会对数字进行分类（你可以试试仅通过   7像素×7像素的窗口观察图像来识别其中的数字）。我们需要让最后一个卷积层的特征包含输入的整体信息。  

最后一层的特征图对每个样本共有   22×22×64=30  976个元素。这太多了。如果你将其展平并在上面添加一个大小为  512的Dense层，那一层将会有     1580万个参数。这对于这样一个小模型来说太多了，会导致严重的过拟合。   

简而言之，使用下采样的原因，一是减少需要处理的特征图的元素个数，二是通过让连续卷积层的观察窗口越来越大（即窗口覆盖原始输入的比例越来越大），从而引入空间过滤器的层级结构。   


是否可以选取平均值，最小值？  

可以，但一般情况下最大值效果最好。   

原因在于特征中往往编码了某种模式或概念在特征图的不同位置是否存在（因此得名特征图），而观察不同特征的最大值而不是平均值能够给出更多的信息。因此，最合理的子采样策略是首先生成密集的特征图（通过无步进的卷积），然后观察特征每个小图块上的最大激活，而不是查看输入的稀疏窗口（通过步进卷积）或对输入图块取平均，因为后两种方法可能导致错过或淡化特征是否存在的信息。